In [2]:
def draw_line(x0, y0, x1, y1):
  dx = abs(x1 - x0)
  dy = abs(y1 - y0)
  if x0 < x1: sx = 1
  else: sx = -1
  if y0 < y1: sy = 1
  else: sy = -1
  err = dx - dy
  x = x0
  y = y0
  coords = []
  while x != x1 or y != y1:
    coords.append([x, y])
    e2 = 2 * err
    if e2 > -dy:
      err -= dy
      x += sx
    if e2 < dx:
      err += dx
      y += sy
  coords.append([x1, y1])
  return coords

# Test
x1 = 100
y1 = 110
x2 = 125
y2 = 120

print(draw_line(x1, y1, x2, y2))

def draw_circle(xc, yc, r):
  x = r
  y = 0
  d = 1 - r
  coords = []
  while x >= y:
    coords.append([xc + x, yc + y])
    coords.append([xc - x, yc + y])
    coords.append([xc + x, yc - y])
    coords.append([xc - x, yc - y])
    coords.append([xc + y, yc + x])
    coords.append([xc - y, yc + x])
    coords.append([xc + y, yc - x])
    coords.append([xc - y, yc - x])
    if d > 0:
      x -= 1
      d = d - 2*x + 1
    y += 1
    d = d + 2*y + 1
  return coords

r = 25
print(draw_circle(x1, y1, r))

[[100, 110], [101, 110], [102, 111], [103, 111], [104, 112], [105, 112], [106, 112], [107, 113], [108, 113], [109, 114], [110, 114], [111, 114], [112, 115], [113, 115], [114, 116], [115, 116], [116, 116], [117, 117], [118, 117], [119, 118], [120, 118], [121, 118], [122, 119], [123, 119], [124, 120], [125, 120]]
[[125, 110], [75, 110], [125, 110], [75, 110], [100, 135], [100, 135], [100, 85], [100, 85], [125, 111], [75, 111], [125, 109], [75, 109], [101, 135], [99, 135], [101, 85], [99, 85], [125, 112], [75, 112], [125, 108], [75, 108], [102, 135], [98, 135], [102, 85], [98, 85], [125, 113], [75, 113], [125, 107], [75, 107], [103, 135], [97, 135], [103, 85], [97, 85], [125, 114], [75, 114], [125, 106], [75, 106], [104, 135], [96, 135], [104, 85], [96, 85], [125, 115], [75, 115], [125, 105], [75, 105], [105, 135], [95, 135], [105, 85], [95, 85], [124, 116], [76, 116], [124, 104], [76, 104], [106, 134], [94, 134], [106, 86], [94, 86], [124, 117], [76, 117], [124, 103], [76, 103], [107, 13

In [21]:
from PIL import Image, ImageOps
import os.path, sys
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.animation import FuncAnimation
from IPython import display

path = "../img/{}/29/"
dirs = sorted(os.listdir(path.format("original")))
Spos = np.array([526, 820])
Rpos = np.array([537, 445])
R = int(np.sqrt(np.sum(np.power(Spos - Rpos, 2))))
seringe = [[254, 673],  # [(x1, y1),
           [380, 687]]  #  (x2, y2)]
a0, af = np.arctan2([seringe[0][1] - R, seringe[1][1] - R], # [y1, y2],
                    [seringe[0][0] - R, seringe[1][0] - R]) # [x1, x2]
def intensity():
  intsty = np.zeros((len(dirs), R))
  plt.figure(facecolor="white", figsize=(15,8))
  for k, item in enumerate(dirs):
    fullpath = os.path.join(path.format("original"), item)
    g = os.path.join(path.format("cropped"), os.path.splitext(item)[0])
    h = os.path.join(path.format("center"), os.path.splitext(item)[0])
    if os.path.isfile(fullpath):
      im = Image.open(fullpath)
      imCrop = im.crop((Spos[0] - R, Spos[1] - R,
                        Spos[0] + R, Spos[1] + R))
      im = imCrop.load()
      imBW = imCrop.convert('L', dither=Image.NONE)
      imBW = ImageOps.invert(imBW)
      imBW.save(g + ".png", "PNG", quality=100)
      imBW = np.asarray(imBW)
      for (t1, t2) in draw_circle(R, R, R - 1):
        t3 = np.arctan2(t2 - R, t1 - R)
        if a0 < t3 or t3 < af:
          for i, (x, y) in enumerate(draw_line(R, R, t1, t2)):
            if i < R: intsty[k][i] += imBW[x, y]
      # Manual Viasualize
      for (x, y) in draw_line(R, R, int(R*(1 + np.cos(a0))),
                              int(R*(1 + np.sin(a0)))):
        im[x, y] = (255, 0, 0)
      for (x, y) in draw_line(R, R, int(R*(1 + np.cos(af))),
                              int(R*(1 + np.sin(af)))):
        im[x, y] = (0, 0, 255)
      for (x, y) in draw_circle(R, R, R - 1):
        t3 = np.arctan2(y - R, x - R)
        if a0 < t3 or t3 < af:
          im[x, y] = (255, 0, 255)
      im[R, R] = (0, 255, 0)
      imCrop.save(h + ".png", "PNG", quality=100)
  norm = np.max(intsty)
  intsty /= norm
  Rs = []
  for k, item in enumerate(dirs):
    f = os.path.join(path.format("analyze"), os.path.splitext(item)[0])
    plt.xlabel("Radius")
    plt.ylabel("Intesity")
    plt.ylim(0, 1)
    plt.xlim(0, R)
    plt.vlines(10, 0, 1, linestyle="dashed")
    plt.title(f"Time Frame: {k}")
    plt.plot(intsty[k])
    if k > 2:
      i = 10
      while intsty[k][i] < 0.5: i += 1
      Rs.append(i)
      plt.plot(i, intsty[k][i], "bo")
    plt.savefig(f + ".pdf", format="pdf")
    plt.clf()
  with open(path.format("analyze") + "data.dat", "w") as fp:
    for x, y in zip(range(3, len(Rs) + 3), Rs):
      print(f"{x}, {y}", file=fp)
intensity()

<Figure size 1080x576 with 0 Axes>